Libraries

In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

In [6]:
# Paths
DATASET_PATH = "E:/Brain Station 23/Human Action Recognition/UCF101/UCF-101"  # Path to UCF101 dataset
FRAMES_PATH = "extracted_frames"  # Path to save extracted frames
PROCESSED_DATA_PATH = "processed_data"  # Path to save processed ROIs

In [7]:
#Initialize YOLOv8 model
model = YOLO('yolov8s.pt')  # You can use other variants like yolov8n, yolov8m, etc.

100%|██████████| 21.5M/21.5M [03:49<00:00, 98.6kB/s]


In [1]:


# Function to extract frames from videos
def extract_frames(video_path, output_dir, frame_rate=10):
    """Extract frames from a video at a fixed frame rate."""
    os.makedirs(output_dir, exist_ok=True)
    video = cv2.VideoCapture(video_path)
    frame_count = 0
    success, frame = video.read()
    while success:
        if frame_count % frame_rate == 0:
            frame_file = os.path.join(output_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_file, frame)
        frame_count += 1
        success, frame = video.read()
    video.release()

# Function to detect humans and crop ROIs using YOLOv8
def detect_humans_yolo_v8(yolo_model, frames_path, output_dir):
    """Detect humans in frames using YOLOv8 and save cropped ROIs."""
    os.makedirs(output_dir, exist_ok=True)
    for frame_file in tqdm(os.listdir(frames_path)):
        frame_path = os.path.join(frames_path, frame_file)
        frame = cv2.imread(frame_path)
        results = yolo_model.predict(frame, conf=0.5)  # Run YOLO detection
        for i, bbox in enumerate(results[0].boxes.xyxy.cpu().numpy()):
            cls = int(results[0].boxes.cls[i])
            conf = results[0].boxes.conf[i]
            if cls == 0 and conf > 0.5:  # Class 0: Human
                x1, y1, x2, y2 = map(int, bbox)
                roi = frame[y1:y2, x1:x2]
                roi_path = os.path.join(output_dir, f"roi_{i}_{frame_file}")
                cv2.imwrite(roi_path, roi)

# Function to prepare temporal sequences
def prepare_temporal_data(dataset_path, sequence_length=16):
    """Prepare sequences of ROIs for temporal model training."""
    data, labels = [], []
    for action_class in os.listdir(dataset_path):
        action_path = os.path.join(dataset_path, action_class)
        if not os.path.isdir(action_path):
            continue
        for video_file in os.listdir(action_path):
            video_path = os.path.join(action_path, video_file)
            video_frames_path = os.path.join(FRAMES_PATH, action_class, video_file.split('.')[0])
            rois_path = os.path.join(PROCESSED_DATA_PATH, action_class, video_file.split('.')[0])

            # Extract frames and detect humans
            extract_frames(video_path, video_frames_path)
            detect_humans_yolo_v8(model, video_frames_path, rois_path)

            # Create sequences
            rois = sorted(os.listdir(rois_path))  # Sort to maintain temporal order
            for i in range(0, len(rois) - sequence_length, sequence_length):
                sequence = []
                for j in range(sequence_length):
                    roi_path = os.path.join(rois_path, rois[i + j])
                    roi = cv2.imread(roi_path)
                    roi = cv2.resize(roi, (224, 224)) / 255.0  # Normalize ROI
                    sequence.append(roi)
                data.append(np.array(sequence))
                labels.append(action_class)  # Label is the folder name
    return np.array(data), np.array(labels)

# Run Preprocessing
X, y = prepare_temporal_data(DATASET_PATH)
np.save("X.npy", X)  # Save sequences
np.save("y.npy", y)  # Save labels


NameError: name 'DATASET_PATH' is not defined